In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

In [ ]:
df = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\heart.csv')
df_XGB = df.copy()
df_RF = df.copy()
df_GNB = df.copy()

# XGBoost

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(df_XGB.drop('output', axis=1), df_XGB['output'], random_state=42)

XGBModel = XGBClassifier()

XGB_scores = cross_val_score(XGBModel, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

In [ ]:
XGBModel.fit(train_X, train_y)
pred = XGBModel.predict(test_X)
accuracy_score(test_y, pred)

# XGB TUNING

In [ ]:
XGBParam1 = {
    'max_depth': [2,4,6,8,10],
    'min_child_weight': [2,4,6,8]
}

XGB_Grid1 = GridSearchCV(XGBClassifier(verbosity=0,
                                       gamma=0,
                                       subsample=0.6,
                                       learning_rate=0.1,
                                       n_estimators=200), XGBParam1)
XGB_Grid1.fit(train_X, train_y)
print(XGB_Grid1.best_params_, XGB_Grid1.best_score_)

In [ ]:
XGBParam2 = {
    'gamma':[0.0,0.1,0.2,0.3,0.4,0.5]
}

XGB_Grid2 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=2,
                                       min_child_weight=4,
                                       subsample=0.6,
                                       learning_rate=0.1,
                                       n_estimators=200), XGBParam2)
XGB_Grid2.fit(train_X, train_y)
print(XGB_Grid2.best_params_, XGB_Grid2.best_score_)

In [ ]:
XGBParam3 = {
    'subsample':[0.5,0.6,0.7]
}

XGB_Grid3 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=2,
                                       min_child_weight=4,
                                       gamma=0.2,
                                       learning_rate=0.04,
                                       n_estimators=200), XGBParam3)
XGB_Grid3.fit(train_X, train_y)
print(XGB_Grid3.best_params_, XGB_Grid3.best_score_)

In [ ]:
XGBParam4 = {
    'learning_rate':[0.03,0.04,0.06]
}

XGB_Grid4 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=2,
                                       min_child_weight=4,
                                       gamma=0.2,
                                       subsample=0.6,
                                       n_estimators=100), XGBParam4)
XGB_Grid4.fit(train_X, train_y)
print(XGB_Grid4.best_params_, XGB_Grid4.best_score_)

In [ ]:
XGBParam5 = {
    'n_estimators':[60, 80, 100]
}

XGB_Grid5 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=2,
                                       min_child_weight=4,
                                       gamma=0.2,
                                       subsample=0.6,
                                       learning_rate=0.04), XGBParam5)
XGB_Grid5.fit(train_X, train_y)
print(XGB_Grid5.best_params_, XGB_Grid5.best_score_)

In [ ]:
XGBModel = XGBClassifier(verbosity=0,
                         max_depth=2,
                         min_child_weight=4,
                         gamma=0.2,
                         subsample=0.6,
                         learning_rate=0.04,
                         n_estimators=100)
XGB_scores = cross_val_score(XGBModel, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

In [ ]:
XGBModel.fit(train_X, train_y)
pred = XGBModel.predict(test_X)
accuracy_score(test_y, pred)

# RandomForestClassifier

In [ ]:
df_RF.head()

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(df_RF.drop('output', axis=1), df_RF['output'], random_state=42)

RFModel = RandomForestClassifier()

RF_scores = cross_val_score(RFModel, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

In [ ]:
RFModel.fit(train_X, train_y)
pred = RFModel.predict(test_X)
accuracy_score(test_y, pred)

In [ ]:
RFParam = {
           "max_depth":[7.5, 10, 12.5, 15],
           "n_estimators":[100, 150, 200],
            "max_samples": [.1,.2,.3]
        }

RF_Grid = GridSearchCV(RandomForestClassifier(), RFParam)
RF_Grid.fit(train_X, train_y)

In [ ]:
grid_results = pd.DataFrame(RF_Grid.cv_results_)
grid_results

In [ ]:
RFModel = RandomForestClassifier(max_depth=12.5, max_samples=0.1, n_estimators=100)

In [ ]:
RF_scores = cross_val_score(RFModel, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

In [ ]:
RFModel.fit(train_X, train_y)
pred = RFModel.predict(test_X)
accuracy_score(test_y, pred)

# GaussianNB

In [ ]:
df_GNB.head()

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(df_GNB.drop('output', axis=1), df_RF['output'], random_state=42)

GNBModel = GaussianNB()

GNB_scores = cross_val_score(GNBModel, train_X, train_y)
(GNB_scores.mean(), GNB_scores.std())

In [ ]:
GNBModel.fit(train_X, train_y)
pred = GNBModel.predict(test_X)
accuracy_score(test_y, pred)

In [ ]:
GNBParam = {
    'var_smoothing': [1e-9, 1e-6, 1e-5]
}

GNB_Grid = GridSearchCV(GaussianNB(), GNBParam)
GNB_Grid.fit(train_X, train_y)

In [ ]:
grid_results = pd.DataFrame(GNB_Grid.cv_results_)
grid_results

In [ ]:
GNBModel = GaussianNB(var_smoothing=1e-5)

In [ ]:
GNB_scores = cross_val_score(GNBModel, train_X, train_y)
(GNB_scores.mean(), GNB_scores.std())

In [ ]:
GNB_Grid.fit(train_X, train_y)
pred = GNB_Grid.predict(test_X)
accuracy_score(test_y, pred)